In [12]:
import pandas as pd
import numpy as np

def process_gesture_data(file_path):
    # 读取 CSV 文件
    df = pd.read_csv(file_path, header=None)
    
    # 移除第一列（时间列无用）
    df = df.iloc[:, 1:].reset_index(drop=True)
    
    # 识别手势分割行（全 0 行）
    empty_rows = df[(df == 0).all(axis=1)].index.tolist()
    print(f"Detected empty rows at indices: {empty_rows}")
    
    # 分割手势数据
    gestures = []
    start_idx = 0
    for idx in empty_rows + [len(df)]:  # 确保包含最后一个手势
        if start_idx < idx:  # 避免连续空行导致空片段
            gestures.append(df.iloc[start_idx:idx].reset_index(drop=True))
        start_idx = idx + 1
    
    processed_gestures = []
    
    for i, gesture in enumerate(gestures):
        data = gesture.to_numpy()
        print(f"Processing Gesture {i+1} with {data.shape[0]} samples")
        
        # 如果数据多于1000，直接截断
        if data.shape[0] > 1000:
            data = data[:1000]
        
        # 插值到 1000 采样点（如果不足）
        if data.shape[0] < 1000:
            x_old = np.linspace(0, 1, data.shape[0])
            x_new = np.linspace(0, 1, 1000)
            data = np.array([np.interp(x_new, x_old, data[:, j]) for j in range(data.shape[1])]).T
        
        # 切分窗口：100 个点一个窗口，步长 50
        windows = []
        for start in range(0, 1000 - 100 + 1, 50):
            windows.append(data[start:start + 100])
        
        # 转换成 [1, window_num, 100, 10]
        windows = np.array(windows)
        windows = np.expand_dims(windows, axis=0)  # 额外的 batch 维度
        
        # 记录手势编号
        gesture_id = np.full((1, windows.shape[1], 100, 1), i + 1)  # 手势编号从 1 开始
        windows = np.concatenate((gesture_id, windows), axis=-1)  # 拼接到第一个通道
        
        # 存储到列表
        processed_gestures.append(windows)
    
    # 合并所有手势数据
    final_data = np.concatenate(processed_gestures, axis=0)  # [gesture_count, window_num, 100, 11]
    return final_data


In [13]:
# 示例调用
file_path = r"new_collect\fzh\sensor_data2.csv"
gesture_data = process_gesture_data(file_path)
print("Final data shape:", gesture_data.shape)

Detected empty rows at indices: []
Processing Gesture 1 with 4009 samples
Final data shape: (1, 19, 100, 11)
